# Covered Topics

### Machine Learning Topics:
- Regression vs Classification
- Missing Values
- Categorical Data
- Pipelines
- Metrics
- Natural Language Processing
  1. Text Processing
    - Cleaning
    - Normalization
    - Tokenization
    - Stop Word Removal
    - Stemming and Lemmatization
  2. Feature Extraction
    - Bag of Words
    - TF-IDF
- Advanced Topics

### Guide
[Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning)

# Regression vs Classification

Regression and classification are categorized under the same umbrella of supervised machine learning. Both share the same concept of utilizing known datasets (referred to as training datasets) to make predictions.

In machine learning, **regression** algorithms attempt to estimate the mapping function (f) from the input variables (x) to numerical or **continuous** output variables (y).
- continuous range: [0, 0.1, 0.2, ...]

On the other hand, **classification** algorithms attempt to estimate the mapping function (f) from the input variables (x) to discrete or **categorical** output variables (y).
- discrete range: [0,1,2]

# Regression Example 1

You will work with data from the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course) to predict home prices in Iowa using 79 explanatory variables describing (almost) every aspect of the homes. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# Read the data
X = pd.read_csv('data/train.csv', index_col='Id')
X_test_full = pd.read_csv('data/test.csv', index_col='Id')

X

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [3]:
# Remove rows with missing target
X.dropna(axis=0, subset=['SalePrice'], inplace=True)

In [4]:
# Separate target from predictors
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

In [5]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [6]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [7]:
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

### Categorical Data

Numbers or strings representing different things.

Survey Question: How often you eat breakfast?
<br>Responses:
- Never
- Rarely
- Most days
- Every day



Survey Question: On a scale of 0 to 2 (0 meaning bad, 1 meaning neutral, 2 meaning good), how was the course?
<br>Responses:
- 0
- 1
- 2

Survey Question: What type of car do you drive?
<br>Responses:
- Toyota
- Honda
- Ford

Three Approaches to handle Categorical Data:
1. Drop Categorical Variables
2. Label Encoding
 - This approach assumes an ordering of the categories: "Never" (0) < "Rarely" (1) < "Most days" (2) < "Every day" (3).
 - This assumption makes sense in this example, because there is an indisputable ranking to the categories. Not all categorical variables have a clear ordering in the values, but we refer to those that do as **ordinal** variables. We refer to categorical variables without an intrinsic ranking as **nominal** variables.
3. One-hot encoding
  - creates new columns indicating the presence (or absence) of each possible value in the original data.

### Missing Values

Three Approaches:
1. Drop Columns with Missing Values
2. Imputation
  - Imputation fills in the missing values with some number.
  - The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than you would get from dropping the column entirely.
3. Imputation and Add Columns Denoting Missing Status

### Pipelines

**Pipelines** are a simple way to keep your data preprocessing and modeling code organized. Specifically, a pipeline bundles preprocessing and modeling steps so you can use the whole bundle as if it were a single step.

We use the `ColumnTransformer` class to bundle together different preprocessing steps. The code below:

- imputes missing values in **numerical** data, and
- imputes missing values and applies a one-hot encoding to **categorical** data.

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [9]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [10]:
def evaluate(predictions, y_valid):
    errors = abs(predictions - y_valid)
    mape = 100 * np.mean(errors / y_valid)
    accuracy = 100 - mape
    print('Model Performance')
    print('MAE: {}'.format(mean_absolute_error(y_valid, preds)))
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [11]:
# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

evaluate(preds, y_valid)

Model Performance
MAE: 17861.780102739725
Average Error: 17861.7801 degrees.
Accuracy = 90.13%.


90.12724117844509

### Improve the model
Now that you've trained a default model as baseline, it's time to tinker with the parameters, to see if we can get better performance! 

A good place is the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) on the random forest in Scikit-Learn. This tells us the most important settings are the number of trees in the forest (n_estimators) and the number of features considered for splitting at each leaf node (max_features).

Also, try other imputing strategies dependent on the data itself.

In [12]:
from pprint import pprint 

In [13]:
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(model.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}


In [14]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=400,max_depth=70,random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

evaluate(preds, y_valid)

Model Performance
MAE: 17282.303193493153
Average Error: 17282.3032 degrees.
Accuracy = 90.38%.


90.37894472420624

# Regression Example 2

In [15]:
# import dataset
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

import matplotlib.pyplot as plt  
import pandas as pd
import numpy as np

%matplotlib inline


In [16]:
# load data
data = load_boston()
# load into a dataframe
df = pd.DataFrame(data['data'], columns=data['feature_names'].tolist())
df['label'] = data['target']
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,label
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


`label` on the right represents **median value of owner-occupied homes in $1000’s average home value in thousands of dollars in each area**. The idea behind this dataset is to use values in the other columns to predict average home value.

In [17]:
X = df.drop("label", axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [18]:
# train model
model = LinearRegression()
model.fit(X_train, y_train)

# predict results
y_pred = model.predict(X_test)

### Regression Metrics
3 of the most common metrics for evaluating predictions on regression machine learning problems:

1. Mean Absolute Error
  - The Mean Absolute Error (or MAE) is the average of the absolute differences between predictions and actual values. It gives an idea of how wrong the predictions were.
  - The measure gives an idea of the magnitude of the error, but no idea of the direction (e.g. over or under predicting).
  - A value of 0 indicates no error or perfect predictions.
2. Mean Squared Error
  - The Mean Squared Error (or MSE) is much like the mean absolute error in that it provides a gross idea of the magnitude of error.
  - Taking the square root of the mean squared error converts the units back to the original units of the output variable and can be meaningful for description and presentation. This is called the Root Mean Squared Error (or RMSE).
  - A value of 0 indicates no error or perfect predictions.
3. R$^2$
  - The R$^2$ (or R Squared) metric provides an indication of the goodness of fit of a set of predictions to the actual values. In statistical literature, this measure is called the coefficient of determination.
  - This is a value between 0 and 1 for no-fit and perfect fit respectively.
  - Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of y, disregarding the input features, would get a R^2 score of 0.0.

[Displaying Accuracy](https://towardsdatascience.com/a-beginners-guide-to-linear-regression-in-python-with-scikit-learn-83a8f7ae2b4f)

In [19]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R^2:', metrics.r2_score(y_test, y_pred)) 

Mean Absolute Error: 3.4584402888845527
Mean Squared Error: 21.199075191484084
Root Mean Squared Error: 4.604245344405974
R^2: 0.7508788069548735


In [20]:
coeff_df = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])  
coeff_df

,Coefficient
CRIM,-0.117263
ZN,0.052823
INDUS,0.027355
CHAS,2.914892
NOX,-16.532906
RM,3.613250
AGE,-0.004996
DIS,-1.552619
RAD,0.366545
TAX,-0.015397


This means that for a unit increase in `NOX`, there is a decrease of 16.99 units in the median value of owner-occupied homes. Similarly, a unit increase in `RM` results in an increase of 4.07 units in the median value of owner-occupied homes. We can see that other features closer to 0 have very little effect on the median value of owner-occupied homes.

# Classification Example 1

The data is from [Heart Disease UCI on Kaggle](https://www.kaggle.com/ronitf/heart-disease-uci). This database contains 76 attributes, but all published experiments refer to using a subset of 14 of them. In particular, the Cleveland database is the only one that has been used by ML researchers to this date. The "goal" field refers to the presence of heart disease in the patient. It is integer valued from 0 (no presence) to 4.

### Guide
[User tentotheminus9 Exploratory Notebook](https://www.kaggle.com/tentotheminus9/what-causes-heart-disease-explaining-the-model)

In [21]:
dt = pd.read_csv("data/heart.csv")
dt.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


Let's change the column names for better understanding

In [22]:
dt.columns = ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate_achieved',
       'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']

In [23]:
dt.head(2)

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1


As seen above, there are categorical columns like sex that are represented with numbers. Let's change them

In [24]:
# Notice below that there are two ways of doing the below commands
# The commented code produces a SettingwithCopyWarning
# Read more here: https://www.dataquest.io/blog/settingwithcopywarning/

# dt[dt['sex'] == 0]['sex'] = 'female'
dt.loc[dt['sex'] == 0, 'sex'] = 'female'

# dt[dt['sex'] == 1]['sex'] = 'female'
dt.loc[dt['sex'] == 1, 'sex'] = 'male'

In [25]:
dt['sex'] = dt['sex'].astype('object')
dt['chest_pain_type'] = dt['chest_pain_type'].astype('object')
dt['fasting_blood_sugar'] = dt['fasting_blood_sugar'].astype('object')
dt['rest_ecg'] = dt['rest_ecg'].astype('object')
dt['exercise_induced_angina'] = dt['exercise_induced_angina'].astype('object')
dt['st_slope'] = dt['st_slope'].astype('object')
dt['thalassemia'] = dt['thalassemia'].astype('object')

In [26]:
dt.dtypes

age                          int64
sex                         object
chest_pain_type             object
resting_blood_pressure       int64
cholesterol                  int64
fasting_blood_sugar         object
rest_ecg                    object
max_heart_rate_achieved      int64
exercise_induced_angina     object
st_depression              float64
st_slope                    object
num_major_vessels            int64
thalassemia                 object
target                       int64
dtype: object

In this case, `object` means string types.

In [27]:
# Remove rows with missing target
dt.dropna(axis=0, subset=['target'], inplace=True)

In [28]:
# Separate target from predictors
y = dt.target              
X = dt.drop(['target'], axis=1)

In [29]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [30]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [31]:
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestClassifier(max_depth=5, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
def evaluate_classifier(model, X_test, y_test):
    '''
    Prints labels, the confusion matrix, and accuracy of the given model
    Parameters:
        model (Pipeline):A pipeline for the input data to follow before the classifier.
        X_test (Array):The array to be used for predictions.
        y_test (Array):The array to be used for comparisons.
    '''
   
    y_pred = model.predict(X_test)
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [36]:
# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

evaluate_classifier(clf, X_valid, y_valid)

Labels: [0 1]
Confusion Matrix:
 [[23  4]
 [ 2 32]]
Accuracy: 0.9016393442622951


![Confusion matrix](https://2.bp.blogspot.com/-EvSXDotTOwc/XMfeOGZ-CVI/AAAAAAAAEiE/oePFfvhfOQM11dgRn9FkPxlegCXbgOF4QCLcBGAs/s1600/confusionMatrxiUpdated.jpg)

# Classification Example 2

In this example, you'll be introduced to some of the steps involved in a Natural Language Processing pipeline:

1. Text Processing
  - Cleaning
  - Normalization
  - Tokenization
  - Stop Word Removal
  - Stemming and Lemmatization
2. Feature Extraction
  - Bag of Words
  - TF-IDF
3. Modeling

### Guide
[Data Scientist Nanodegree](https://github.com/kevinlwebb/Nanodegrees/blob/master/Data%20Scientist%20for%20Enterprise/3%20-%20Data%20Engineering/3.3%20NLP%20Pipelines.ipynb)

In [37]:
import re
import nltk

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC

/Users/kevinwebb/Desktop/env/ds/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/kevinwebb/Desktop/env/ds/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, defaultdict
/Users/kevinwebb/Desktop/env/ds/lib/python3.7/site-packages/nltk/lm/vocabulary.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Counter, Iterable


In [38]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/kevinwebb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kevinwebb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kevinwebb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [40]:
url = "data/tweet_train.csv"
df = pd.read_csv(url)

In [41]:
df.head(5)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [42]:
df['cleaned_tweet'] = df.tweet.apply(lambda x: ' '.join([word for word in x.split() if not word.startswith('@')]))
df['cleaned_tweet'] = df.cleaned_tweet.str.replace("#","")

In [43]:
df.head(5)

,id,label,tweet,cleaned_tweet
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i can't use cause they ...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in urð...
4,5,0,factsguide: society now #motivation,factsguide: society now motivation


In [44]:
X = df.cleaned_tweet
Y = df.label

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

### Normalization
- Lower case text
- Replace characters that are not a-z, A-Z, or numbers with a space

### Tokenization
Separating words into different elements in a list

### Stop Word Removal
Removing unimportant, common words
- I, me, myself, etc
  
### Stemming
Process of reducing a word to its stem or root form. This eliminates the letters at the end.
- branch: branches, branching, branched

### Lemmatization
Process of reducing words to a normalized form. This uses a dicionary of variants to roots.
- be: is, was, were

In [45]:
def tokenize(text):
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word).lower().strip() for word in tokens if word.lower().strip() not in stop_words]

    return tokens

### Bag of Words
Feature representation

Treats each document as an unordered collection or bag of words

Useful approach: turn each document unto a vector of numbers representing how many times each word occurs in a document

### TF-IDF
Con of Bag-of-Words: It treats every words as being equally important
Finding frequency of words (document frequency)

Using the product of two weights: term frequency and inverse document frequency

Below, we'll use two useful methods of vectorizing text, and there's one that combines the power of both:
- `CountVectorizer` - Bag of Words
- `TfidfTransformer` - TF-IDF values
- `TfidfVectorizer` - Bag of Words AND TF-IDF values

In [46]:
clf = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
])

clf.fit(X_train, Y_train)

evaluate_classifier(clf, X_test, Y_test)

Labels: [0 1]
Confusion Matrix:
 [[5924   31]
 [ 183  255]]
Accuracy: 0.9665258876896605


# Advanced Topics

### [Cross Validation](https://www.kaggle.com/alexisbcook/cross-validation)

In cross-validation, we run our modeling process on different subsets of the data to get multiple measures of model quality.

Cross-validation gives a more accurate measure of model quality, which is especially important if you are making a lot of modeling decisions. However, it can take longer to run, because it estimates multiple models (one for each fold).

### [XGBoost](https://www.kaggle.com/alexisbcook/xgboost)

We have made predictions with the random forest method, which achieves better performance than a single decision tree simply by averaging the predictions of many decision trees.

[XGBoost](https://xgboost.readthedocs.io/en/latest/) is a the leading software library for working with standard tabular data (the type of data you store in Pandas DataFrames, as opposed to more exotic types of data like images and videos). With careful parameter tuning, you can train highly accurate models.

**XGBoost** stands for extreme gradient boosting, which is an implementation of gradient boosting with several additional features focused on performance and speed. (Scikit-learn has another version of gradient boosting, but XGBoost has some technical advantages.)

### [TensorFlow](https://www.tensorflow.org/)
TensorFlow is more of a low-level library; basically, we can think of TensorFlow as the Lego bricks (similar to NumPy and SciPy) that we can use to implement machine learning algorithms whereas scikit-learn comes with off-the-shelf algorithms, e.g., algorithms for classification such as SVMs, Random Forests, Logistic Regression, and many, many more. TensorFlow really shines if we want to implement deep learning algorithms, since it allows us to take advantage of GPUs for more efficient training. Comparison [link](https://sebastianraschka.com/faq/docs/tensorflow-vs-scikitlearn.html)